# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../project.env")
sys.path.append(os.environ["PYTHONPATH"])

import pickle

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAINheader as train



Chosen class grouping: alpha-classes


Directory /Users/diego/Desktop/iteso/TOG/ exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/src exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/media exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/scores exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/alpha-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH1/alpha-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH2/alpha-classes exists. Continuing with execution
Directory /Users/diego/Desktop/iteso/TOG/data/PH3/alpha-classes exists. Continuing with execution
d

In [3]:
TRAIN_classes = 'alpha-classes'

# Model training

## KNN

### Active Hand prediction

In [4]:
train.arch.find_best(sup.DATA_AH_PF, sup.active_hand_col, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.9067193675889328
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9297879985627021
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 3}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9347466762486525
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'AHpf', 'label_col': 'active_hand', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 5}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/AHpf exists. Continuing with execution
updating best... 0.9373338124326267
	

### Sign prediction

In [5]:
train.arch.find_best(sup.DATA_S_PF, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.06497175141242938
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spf exists. Continuing with execution
updating best... 0.06581920903954802
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 6}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spf exists. Continuing with execution
updating best... 0.06850282485875706
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 8}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spf exists. Continuing with execution
updating best... 0.071751412429378

In [6]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_KNN_CODE)

updating best... 0.16610169491525423
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 1}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spv does not exist. Creating it and continuing with execution
updating best... 0.1694915254237288
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 13}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spv exists. Continuing with execution
updating best... 0.17966101694915254
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'alpha-classes'}
	{'arch': 'KNN', 'k': 14}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/KNN/Spv exists. Continuing with execution
updating 

## BERT

In [7]:
train.arch.find_best(sup.DATA_S_PV, sup.class_numeric_column, TRAIN_classes, sup.TRAIN_BERT_CODE)

updating best... 0.03728813559322034
	{'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 1, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'alpha-classes', 'batch_size': 1024, 'input_dim': 1}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 2}
Directory /Users/diego/Desktop/iteso/TOG/bin/gen/TRAIN/alpha-classes/BERT/Spv/prajjwal1/bert-tiny does not exist. Creating it and continuing with execution
updating best... 0.04915254237288136
	{'PH2': True, 'PH3': True, 'reducer': 'umap', 'kernel': '', 'n': 4, 'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'alpha-classes', 'batch_size': 1024, 'input_dim': 4}
	{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'prajjwal1/bert-tiny', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 2e-05, 'loss_fn': <class 'torch.nn.modul

# Score saving

In [8]:
import pandas as pd
from datetime import datetime

## KNN

In [9]:
knn_scores_df = pd.DataFrame(sup.knn_score_tracker, columns=sup.knn_scores_columns)

In [10]:
knn_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
0,alpha-classes,0.906719,AHpf,True,True,pca,None,1,1
1,alpha-classes,0.898742,AHpf,True,True,pca,None,1,2
2,alpha-classes,0.929788,AHpf,True,True,pca,None,1,3
3,alpha-classes,0.926626,AHpf,True,True,pca,None,1,4
4,alpha-classes,0.934747,AHpf,True,True,pca,None,1,5
...,...,...,...,...,...,...,...,...,...
3865,alpha-classes,0.374576,Spv,False,False,None,None,-1,11
3866,alpha-classes,0.359322,Spv,False,False,None,None,-1,12
3867,alpha-classes,0.352542,Spv,False,False,None,None,-1,13
3868,alpha-classes,0.340678,Spv,False,False,None,None,-1,14


In [11]:
knn_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,k
634,alpha-classes,0.984765,AHpf,True,False,None,None,-1,5
632,alpha-classes,0.984621,AHpf,True,False,None,None,-1,3
635,alpha-classes,0.984549,AHpf,True,False,None,None,-1,6
633,alpha-classes,0.984477,AHpf,True,False,None,None,-1,4
636,alpha-classes,0.984333,AHpf,True,False,None,None,-1,7
1278,alpha-classes,0.984333,AHpf,False,False,None,None,-1,4
1277,alpha-classes,0.984190,AHpf,False,False,None,None,-1,3
637,alpha-classes,0.984118,AHpf,True,False,None,None,-1,8
639,alpha-classes,0.984118,AHpf,True,False,None,None,-1,10
350,alpha-classes,0.983974,AHpf,True,True,umap,None,4,6


In [12]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE))
knn_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_KNN_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/alpha-classes/KNN does not exist. Creating it and continuing with execution


## BERT

In [13]:
bert_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)

In [14]:
bert_scores_df

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
0,alpha-classes,0.037288,Spv,True,True,pca,None,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
1,alpha-classes,0.037288,Spv,True,True,kpca,poly,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
2,alpha-classes,0.037288,Spv,True,True,kpca,rbf,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
3,alpha-classes,0.037288,Spv,True,True,kpca,sigmoid,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
4,alpha-classes,0.037288,Spv,True,True,kpca,cosine,1,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,alpha-classes,0.035593,Spv,False,True,kpca,rbf,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
82,alpha-classes,0.038983,Spv,False,True,kpca,sigmoid,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
83,alpha-classes,0.045763,Spv,False,True,kpca,cosine,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
84,alpha-classes,0.030508,Spv,False,True,umap,None,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [15]:
bert_scores_df.sort_values(by="accuracy", ascending=False).head(50)

,class_list,accuracy,data_unit,PH2,PH3,reducer,kernel,n,loadble,lr,optimizer,loss_fn,num_epochs
37,alpha-classes,0.062712,Spv,True,True,kpca,poly,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
56,alpha-classes,0.050847,Spv,False,True,kpca,poly,3,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
23,alpha-classes,0.049153,Spv,True,True,umap,None,4,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
83,alpha-classes,0.045763,Spv,False,True,kpca,cosine,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
30,alpha-classes,0.044068,Spv,True,True,pca,None,6,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
67,alpha-classes,0.044068,Spv,False,True,pca,None,5,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
54,alpha-classes,0.042373,Spv,False,True,umap,None,2,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
62,alpha-classes,0.040678,Spv,False,True,kpca,poly,4,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
39,alpha-classes,0.038983,Spv,True,True,kpca,sigmoid,7,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2
66,alpha-classes,0.038983,Spv,False,True,umap,None,4,prajjwal1/bert-tiny,0.00002,AdamW (\nParameter Group 0\n amsgrad: False...,CrossEntropyLoss(),2


In [16]:
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE))
bert_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, sup.TRAIN_BERT_CODE, f"{now}.csv"), index=False)

Directory /Users/diego/Desktop/iteso/TOG/scores/alpha-classes/BERT does not exist. Creating it and continuing with execution
